# Skill estimation using graphical models

Let's see how we can use a simple model to predict a hidden "skill level" of players based on their performance in a collection of games against each other.  We need data (the games and outcomes), and a model of how skill translates into these outcomes (e.g., higher skilled players have a better chance of winning).

In [1]:
import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline         

### A simple list of games & outcomes

In [2]:
games = [
    (0,2, +1),  # P0 played P2 & won
    (0,2, +1),  # played again, same outcome
    (1,2, -1),  # P1 played P2 & lost
    (0,1, -1),  # P0 played P1 and lost
#     (1,2, +1),  # P1 played P2 & lost
#     (1,2, +1),  # P1 played P2 & lost
#     (1,2, +1),  # P1 played P2 & lost
#     (1,2, +1),  # P1 played P2 & lost
#     (1,2, +1),  # P1 played P2 & lost
#     (1,2, +1),  # P1 played P2 & lost
#     (1,2, +1),  # P1 played P2 & lost
#     (1,2, +1),  # P1 played P2 & lost
#     (0,2, +1),
#     (0,2, +1),
#     (0,2, +1),
#     (0,2, +1),
#     (0,2, +1),
#     (0,2, +1),
#     (0,2, +1),
#     (0,2, +1),
#     (1,0, +1),
#     (1,0, +1),
#     (1,0, +1),
#     (1,0, +1),
#     (1,0, +1),
#     (1,0, +1),
#     (1,0, +1),
]

### Win probability and graphical model

In [3]:
nplayers = max( [max(g[0],g[1]) for g in games] )+1 #number of players
nlevels = 10   # let's say 10 discrete skill levels
scale = .3     # this scales how skill difference translates to win probability

# Make variables for each player; value = skill level
X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)   

# Information from each game: what does Pi winning over Pj tell us?
#    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

# before any games, uniform belief over skill levels for each player:
factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

# Now add the information from each game:
for g in games:
    P1,P2,win = g[0],g[1],g[2]
    if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
    factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

In [4]:
model = gm.GraphModel(factors)
model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

In [5]:
if model.nvar < 0:       # for very small models, we can do brute force inference:
    jt = model.joint()
    jt /= jt.sum()       # normalize the distribution and marginalize the table
    bel = [jt.marginal([i]) for i in range(nplayers)] 

else:                    # otherwise we need to use some approximate inference:
    from pyGM.messagepass import LBP, NMF
    lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    #lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

Iter 1: -2.471684774393812
Iter 2: -3.006255512017563
Iter 3: -3.1777481521503583
Iter 4: -3.205567636371387
Iter 5: -3.2108508120415324
Iter 6: -3.2116774755212765
Iter 7: -3.2118010272359196
Iter 8: -3.211824198459518
Iter 9: -3.2118278160636624
Iter 10: -3.21182835654903


The normalization constant, $\log(Z)$, represents the (log) probability of evidence for our model, namely the probability of the observed game outcomes given our parameters, etc.  We could experiment with changing the win probability function or its scaling parameter to try to make our model better fit the data using this value.

For example, if you play with "scale" on these toy data, you'll find scale=0 (so that every game is a 50-50 chance independent of skill level) fits the data pretty well, because the few outcomes listed are not really consistent with skill determining outcome.  But, if you change the data so that there is an obvious ordering of skill (e.g., P0 then 1 and/or 2), a larger scale parameter will better fit the data.

###  Ranking players by predicted skill

In [6]:
print("Mean skill estimates: ")
print([ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)] )

Mean skill estimates: 
[5.2757029857915185, 4.5, 3.7242970142084806]


### Predicting match outcomes

In [7]:
i,j = 0,1
print("Estimated probability P{} beats P{} next time:".format(i,j))
# Expected value (over skill of P0, P1) of Pr[win | P0-P1]
if i<j:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() )
else:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum() )
    
# Notes: we should probably use the joint belief over Xi and Xj, but for simplicity
#  with approximate inference we'll just use the estimated singleton marginals

Estimated probability P0 beats P1 next time:
0.547035108623266


## Attempts:

### training_data :

In [8]:
import pandas as pd

In [9]:
train_data=pd.read_csv('train.csv',index_col=False,names=['date', 'p1', 'p1_outcome', 'score', 'p2', 'p2_outcome', 'p1_race', 'p2_race', 'addon', 'type'])

In [10]:
train_data

,date,p1,p1_outcome,score,p2,p2_outcome,p1_race,p2_race,addon,type
0,09/19/2016,MC,[loser],0–2,Stats,[winner],P,P,LotV,online
1,09/19/2016,MC,[loser],1–2,Dark,[winner],P,Z,LotV,online
2,09/13/2016,MC,[loser],0–2,INnoVation,[winner],P,T,LotV,online
3,08/27/2016,MC,[loser],0–1,TRUE,[winner],P,Z,LotV,online
4,07/16/2016,MC,[winner],1–0,Super,[loser],P,P,LotV,offline
...,...,...,...,...,...,...,...,...,...,...
193069,05/04/2013,Keiras,[loser],1–2,Harpner,[winner],Z,P,HotS,online
193070,05/04/2013,Keiras,[winner],2–0,Harpner,[loser],Z,P,HotS,online
193071,04/13/2013,Keiras,[loser],0–1,maTTzour,[winner],Z,T,HotS,online
193072,03/17/2012,Keiras,[loser],1–2,nukestrike,[winner],Z,T,WoL,online


In [11]:
train_data = train_data.reindex(np.random.permutation(train_data.index))
train_data = train_data[:10000]
#train_data

In [12]:
# list of players
players = np.unique(np.concatenate((train_data['p1'], train_data['p2'])))

#map between player names to a unique playerid
playerid = dict()
for i in range(len(players)):
    playerid[players[i]]=i

In [13]:
len(players)

991

In [14]:
#drop other columns for now
train_data.drop(columns=['date','p1_outcome','p2_outcome','p1_race', 'p2_race', 'addon', 'type'], inplace=True)

In [15]:
# replace player names with player id
train_data['p1'].replace(playerid,inplace=True)
train_data['p2'].replace(playerid,inplace=True)

In [16]:
train_data[['p1_win','p2_win']] = train_data.score.str.split("–",expand=True) 
train_data['outcome']=0
train_data.drop(columns=['score'],inplace=True)
train_data

,p1,p2,p1_win,p2_win,outcome
54067,421,804,3,0,0
21417,694,744,2,0,0
38847,979,88,1,0,0
50083,712,285,2,0,0
111019,365,16,2,0,0
...,...,...,...,...,...
65581,478,330,2,1,0
190798,521,854,0,1,0
4598,430,803,1,0,0
24943,281,294,2,0,0


In [17]:
new_train_data=[]
for index,g in train_data.iterrows():
    p1_win, p2_win = int(g['p1_win']),int(g['p2_win'])

    g['outcome']=1
    for i in range(p1_win):
        new_train_data.append(g.copy())
    g['outcome']=-1
    for i in range(p2_win):
        new_train_data.append(g.copy())
    

In [18]:
new_train_data=pd.DataFrame(new_train_data)
new_train_data.drop(columns=['p1_win','p2_win'],inplace=True)
train_data=new_train_data

In [19]:
train_data

,p1,p2,outcome
54067,421,804,1
54067,421,804,1
54067,421,804,1
21417,694,744,1
21417,694,744,1
...,...,...,...
24943,281,294,1
24943,281,294,1
97481,750,983,1
97481,750,983,1


In [20]:
train_games=[tuple(r) for r in train_data.to_numpy()]
train_games

[(421, 804, 1),
 (421, 804, 1),
 (421, 804, 1),
 (694, 744, 1),
 (694, 744, 1),
 (979, 88, 1),
 (712, 285, 1),
 (712, 285, 1),
 (365, 16, 1),
 (365, 16, 1),
 (173, 853, 1),
 (173, 853, -1),
 (173, 853, -1),
 (153, 788, 1),
 (859, 92, -1),
 (48, 439, 1),
 (408, 447, 1),
 (970, 519, -1),
 (970, 519, -1),
 (163, 755, -1),
 (163, 755, -1),
 (98, 712, -1),
 (98, 712, -1),
 (567, 431, 1),
 (567, 431, -1),
 (567, 431, -1),
 (138, 90, -1),
 (787, 130, 1),
 (787, 130, 1),
 (795, 537, -1),
 (795, 537, -1),
 (907, 876, 1),
 (907, 876, -1),
 (907, 876, -1),
 (22, 204, -1),
 (22, 204, -1),
 (569, 635, 1),
 (144, 386, 1),
 (144, 386, 1),
 (144, 386, -1),
 (764, 912, -1),
 (764, 912, -1),
 (718, 274, -1),
 (718, 274, -1),
 (397, 946, -1),
 (837, 497, -1),
 (91, 497, 1),
 (91, 497, 1),
 (216, 298, -1),
 (216, 298, -1),
 (433, 631, 1),
 (271, 665, 1),
 (271, 665, 1),
 (271, 665, -1),
 (119, 159, -1),
 (713, 204, -1),
 (713, 204, -1),
 (669, 637, -1),
 (669, 637, -1),
 (573, 564, 1),
 (573, 564, -1),
 (

### Model:

In [21]:
# nplayers = max( [max(g[0],g[1]) for g in games] )+1 #number of players
nplayers = len(players)
nlevels = 100   # let's say 10 discrete skill levels
scale = .3     # this scales how skill difference translates to win probability

# Make variables for each player; value = skill level
X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)   

# Information from each game: what does Pi winning over Pj tell us?
#    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

# before any games, uniform belief over skill levels for each player:
factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

# Now add the information from each game:
for g in train_games:
    P1,P2,win = g[0],g[1],g[2]
    if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
    factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

In [22]:
model = gm.GraphModel(factors)
model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

In [23]:
if model.nvar < 0:       # for very small models, we can do brute force inference:
    jt = model.joint()
    jt /= jt.sum()       # normalize the distribution and marginalize the table
    bel = [jt.marginal([i]) for i in range(nplayers)] 

else:                    # otherwise we need to use some approximate inference:
    from pyGM.messagepass import LBP, NMF
    #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

Iter 0: -inf


D:\Julia\Jupyter_Workspace\179\project\pyGM\factor.py:578: RuntimeWarning: invalid value encountered in multiply
  op( A.t.reshape(dA,order='A') , B.t.reshape(dB,order='A'), out=out.t )   # TODO: order=A necessary?


ValueError: Non-normalizable factor (perhaps log factor?)

###  Ranking players by predicted skill

In [ ]:
print("Mean skill estimates: ")
print([ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)] )

## Predicting on valid set

In [ ]:
valid_data=pd.read_csv('valid.csv',index_col=False,names=['date', 'p1', 'p1_outcome', 'score', 'p2', 'p2_outcome', 'p1_race', 'p2_race', 'addon', 'type'])

#drop other columns for now
valid_data.drop(columns=['date','score','p2_outcome','p1_race', 'p2_race', 'addon', 'type'], inplace=True)
# replace player names with player id and loser/winner to -1/1
valid_data['p1'].replace(playerid,inplace=True)
valid_data['p2'].replace(playerid,inplace=True)
valid_data['p1_outcome'].replace({"[loser]":-1,"[winner]":+1},inplace=True)

In [ ]:
valid_games=[tuple((r[0],r[2],r[1])) for r in valid_data.to_numpy()]
valid_games

In [ ]:
num_valid_game=len(valid_games)
acc=0

#for g in valid_games:
for g in 1000:
    try:
        i,j,result=int(g[0]),int(g[1]),int(g[2])
    except:
        num_valid_game-=1
        continue
    if i<j:
        prediction = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() 
    else:
        prediction = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum()

    if prediction > 0.5:
        prediction = +1
    else:
        prediction = -1
    acc+=(prediction==result)
    
#print(acc/num_valid_game)
print(acc/1000)